In [1]:
# Import Modules/Libraries
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

%matplotlib inline
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

np.random.seed(1)

import warnings
#suppress warnings with numpy for sigmoid function
warnings.filterwarnings('ignore')

# Logistic Regression

In [2]:
# Load datasets
train = pd.read_csv('../datasets/clean/cleaned_train.csv', index_col = False)
test = pd.read_csv('../datasets/clean/cleaned_test.csv', index_col = False)

FileNotFoundError: [Errno 2] No such file or directory: '../datasets/clean/cleaned_train.csv'

In [ ]:
train.head(3)

In [ ]:
test.head(3)

In [ ]:
#Split X (independent variables) with the target value Y
target_columns = ['Gryffindor', 'Hufflepuff', 'Ravenclaw', 'Slytherin']
X_train = np.array(train.drop(columns = target_columns))
y_train = np.array(train[target_columns])

In [ ]:
print ('The shape of X_train is: ' + str(X_train.shape))
print ('The shape of y_train is: ' + str(y_train.shape))
print ('We have m = %d training examples' % (len(y_train)))

In [ ]:
X_train[0, :]

In [ ]:
# To Access the ith column 
y_train[:, 0]

### Sigmoid function

For logistic regression, the model is represented as

$$ f_{\mathbf{w},b}(x) = g(\mathbf{w}\cdot \mathbf{x} + b)$$
where function $g$ is the sigmoid function. The sigmoid function is defined as:

$$g(z) = \frac{1}{1+e^{-z}}$$

In [ ]:
def sigmoid(z):
    """
    Compute the sigmoid of z
    Args:
        z (ndarray): A scalar, numpy array of any size.
    Returns:
        g (ndarray): sigmoid(z), with the same shape as z
    """
    return 1 / (1 + np.exp(-z))

In [ ]:
print ("sigmoid(0) = " + str(sigmoid(0)))

### Cost function for logistic regression


Recall that for logistic regression, the cost function is of the form 

$$ J(\mathbf{w},b) = \frac{1}{m}\sum_{i=0}^{m-1} \left[ loss(f_{\mathbf{w},b}(\mathbf{x}^{(i)}), y^{(i)}) \right] \tag{1}$$

where
* m is the number of training examples in the dataset


* $loss(f_{\mathbf{w},b}(\mathbf{x}^{(i)}), y^{(i)})$ is the cost for a single data point, which is - 

    $$loss(f_{\mathbf{w},b}(\mathbf{x}^{(i)}), y^{(i)}) = (-y^{(i)} \log\left(f_{\mathbf{w},b}\left( \mathbf{x}^{(i)} \right) \right) - \left( 1 - y^{(i)}\right) \log \left( 1 - f_{\mathbf{w},b}\left( \mathbf{x}^{(i)} \right) \right) \tag{2}$$
    
    
*  $f_{\mathbf{w},b}(\mathbf{x}^{(i)})$ is the model's prediction, while $y^{(i)}$, which is the actual label

*  $f_{\mathbf{w},b}(\mathbf{x}^{(i)}) = g(\mathbf{w} \cdot \mathbf{x^{(i)}} + b)$ where function $g$ is the sigmoid function.
    * It might be helpful to first calculate an intermediate variable $z_{\mathbf{w},b}(\mathbf{x}^{(i)}) = \mathbf{w} \cdot \mathbf{x^{(i)}} + b = w_0x^{(i)}_0 + ... + w_{n-1}x^{(i)}_{n-1} + b$ where $n$ is the number of features, before calculating $f_{\mathbf{w},b}(\mathbf{x}^{(i)}) = g(z_{\mathbf{w},b}(\mathbf{x}^{(i)}))$

In [ ]:
def compute_cost(X, y, w, b, lambda_= 1):
    """
    Computes the cost over all examples
    Args:
      X : (ndarray Shape (m,n)) data, m examples by n features
      y : (array_like Shape (m,)) target value 
      w : (array_like Shape (n,)) Values of parameters of the model      
      b : scalar Values of bias parameter of the model
      lambda_: unused placeholder
    Returns:
      total_cost: (scalar)         cost 
    """

    m, n = X.shape
    
    cost = 0
    for i in range(m):
        z = np.dot(X[i],w) + b
        f_wb = sigmoid(z)
        cost += -y[i]*np.log(f_wb) - (1-y[i])*np.log(1-f_wb)
    total_cost = cost/m
    

    return total_cost

In [ ]:
(1-y_train[:, 2])

In [ ]:
m, n = X_train.shape

# Compute and display cost with w initialized to zeroes
initial_w = np.zeros(n)
initial_b = 0.
cost = compute_cost(X_train, y_train[:, 0], initial_w, initial_b)
print('Cost at initial w (zeros): {:.3f}'.format(cost))

###  Gradient for logistic regression

In this section, you will implement the gradient for logistic regression.

Recall that the gradient descent algorithm is:

$$\begin{align*}& \text{repeat until convergence:} \; \lbrace \newline \; & b := b -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial b} \newline       \; & w_j := w_j -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial w_j} \tag{1}  \; & \text{for j := 0..n-1}\newline & \rbrace\end{align*}$$

where, parameters $b$, $w_j$ are all updated simultaniously

compute $\frac{\partial J(\mathbf{w},b)}{\partial w}$, $\frac{\partial J(\mathbf{w},b)}{\partial b}$ from equations (2) and (3) below.

$$
\frac{\partial J(\mathbf{w},b)}{\partial b}  = \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - \mathbf{y}^{(i)}) \tag{2}
$$
$$
\frac{\partial J(\mathbf{w},b)}{\partial w_j}  = \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - \mathbf{y}^{(i)})x_{j}^{(i)} \tag{3}
$$

In [ ]:
def compute_gradient(X, y, w, b, lambda_=None): 
    """
    Computes the gradient for logistic regression 
 
    Args:
      X : (ndarray Shape (m,n)) variable such as house size 
      y : (array_like Shape (m,1)) actual value 
      w : (array_like Shape (n,1)) values of parameters of the model      
      b : (scalar)                 value of parameter of the model 
      lambda_: unused placeholder.
    Returns
      dj_dw: (array_like Shape (n,1)) The gradient of the cost w.r.t. the parameters w. 
      dj_db: (scalar)                The gradient of the cost w.r.t. the parameter b. 
    """
    m, n = X.shape
    dj_dw = np.zeros(w.shape)
    dj_db = 0.

    z = np.dot(X, w) + b
    f_wb = sigmoid(z)
    
    for j in range(n):
        dj_dw[j] = (np.sum(np.dot(f_wb - y, X.T[j])))
            
    dj_dw = dj_dw / m
    dj_db = np.sum(f_wb - y) / m
        
    return dj_db, dj_dw

### Learning parameters using gradient descent 


In [ ]:
def gradient_descent(X, y, w_in, b_in, cost_function, gradient_function, alpha, num_iters, lambda_): 
    """
    Performs batch gradient descent to learn theta. Updates theta by taking 
    num_iters gradient steps with learning rate alpha
    
    Args:
      X :    (array_like Shape (m, n)
      y :    (array_like Shape (m,))
      w_in : (array_like Shape (n,))  Initial values of parameters of the model
      b_in : (scalar)                 Initial value of parameter of the model
      cost_function:                  function to compute cost
      alpha : (float)                 Learning rate
      num_iters : (int)               number of iterations to run gradient descent
      lambda_ (scalar, float)         regularization constant
      
    Returns:
      w : (array_like Shape (n,)) Updated values of parameters of the model after
          running gradient descent
      b : (scalar)                Updated value of parameter of the model after
          running gradient descent
    """
    
    # number of training examples
    m = len(X)
    
    # An array to store cost J and w's at each iteration primarily for graphing later
    J_history = []
    w_history = []
    
    for i in range(num_iters):

        # Calculate the gradient and update the parameters
        dj_db, dj_dw = gradient_function(X, y, w_in, b_in, lambda_)   

        # Update Parameters using w, b, alpha and gradient
        w_in = w_in - alpha * dj_dw
        b_in = b_in - alpha * dj_db
       
        # Save cost J at each iteration
        if i<100000:      # prevent resource exhaustion 
            cost =  cost_function(X, y, w_in, b_in, lambda_)
            J_history.append(cost)

        # Print cost every at intervals 10 times or as many iterations if < 10
        if i% math.ceil(num_iters/10) == 0 or i == (num_iters-1):
            w_history.append(w_in)
            print(f"Iteration {i:4}: Cost {float(J_history[-1]):8.2f}   ")
        
    return w_in, b_in, J_history, w_history #return w and J,w history for graphing

## Fit 

In [ ]:
m, n = X_train.shape
initial_w = 0.01 * (np.random.rand(n).reshape(-1,1) - 0.5)
initial_b = -8


# Some gradient descent settings
iterations = 10000
alpha = 0.01

w,b, J_history,_ = gradient_descent(X_train ,y_train[:, 3], initial_w, initial_b, 
                                   compute_cost, compute_gradient, alpha, iterations, 0)

## Predict

In [ ]:
def predict(X, w, b): 
    """
    Predict whether the label is 0 or 1 using learned logistic
    regression parameters w
    
    Args:
    X : (ndarray Shape (m, n))
    w : (array_like Shape (n,))      Parameters of the model
    b : (scalar, float)              Parameter of the model

    Returns:
    p: (ndarray (m,1))
        The predictions for X using a threshold at 0.5
    """
    # number of training examples
    m, n = X.shape
    p = np.zeros(m)

    f_wb = sigmoid(np.dot(X, w) + b)
    for i, prob in enumerate(f_wb):
        p[i] = 1 if prob >= 0.50 else 0
    return p

In [ ]:
#Compute accuracy on our training set
p = predict(X_train, w,b)
print('Train Accuracy: %f'%(np.mean(p == y_train[:, 3]) * 100))

----------

# Make a class out of it...

In [ ]:
class LogisticRegression:
    """
        A class to perform Logistic Regression
    """
    def __init__(self):
        """
            Define attributes which will be passed later
        """
        # train data and label
        self.X = None
        self.y = None
        
        # m: number of observations
        self.m = None
        
        # n: number of independent variables (X)
        self.n = None
        
        # store historic value of cost function. init as +infinity
        self.costs = [np.inf]

        self.J_history = []
        self.w_history = []
        
        # parameteres (weights)
        self.w = None
        self.b = None
        
        # regularization constant lambda_ (scalar, float)
        self.lambda_ = None

        
    def sigmoid(self, z):
        """
        Compute the sigmoid of z
        Args:
            z (ndarray): A scalar, numpy array of any size.
        Returns:
            g (ndarray): sigmoid(z), with the same shape as z
        """
        return 1 / (1 + np.exp(-z))

    
    def compute_cost(self, X, y, w, b):
        """
        Computes the cost over all examples
        Args:
          X : (ndarray Shape (m,n)) data, m examples by n features
          y : (array_like Shape (m,)) target value 
          w : (array_like Shape (n,)) Values of parameters of the model      
          b : scalar Values of bias parameter of the model
          lambda_: unused placeholder
        Returns:
          total_cost: (scalar)         cost 
        """

        m, n = X.shape

        z = np.dot(X, w) + b
        pred = self.sigmoid(z)

        pred[pred == 1] = 1-1e-9 #hard cap max threshold
        
        
        cost = np.dot(-y, np.log(pred)) - (np.dot(1 - y, np.log(1 - pred)))
        total_cost = np.sum(cost) / m
    
        reg_cost = np.sum(np.dot(w, w.T))
        total_cost = total_cost + (self.lambda_/(2 * m)) * reg_cost
        
        return total_cost

    
    def compute_gradient(self, X, y, w, b, lambda_): 
        """
        Computes the gradient for logistic regression 

        Args:
          X : (ndarray Shape (m,n)) variable such as house size 
          y : (array_like Shape (m,1)) actual value 
          w : (array_like Shape (n,1)) values of parameters of the model      
          b : (scalar)                 value of parameter of the model 
          lambda_: unused placeholder.
        Returns
          dj_dw: (array_like Shape (n,1)) The gradient of the cost w.r.t. the parameters w. 
          dj_db: (scalar)                The gradient of the cost w.r.t. the parameter b. 
        """
        m, n = X.shape
        dj_dw = np.zeros(w.shape)
        dj_db = 0.0

        z = np.dot(X, w) + b
        f_wb = self.sigmoid(z)

        for j in range(n):
            dj_dw[j] = (np.sum(np.dot(f_wb - y, X.T[j])))

        dj_dw = dj_dw / m
        dj_dw += np.dot((lambda_ / m), w) # add regularization
        dj_db = np.sum(f_wb - y) / m

        return dj_db, dj_dw


    def gradient_descent(self, cost_function, gradient_function, alpha, num_iters, show_every): 
        """
        Performs batch gradient descent to learn theta. Updates theta by taking 
        num_iters gradient steps with learning rate alpha

        Args:
          X :    (array_like Shape (m, n)
          y :    (array_like Shape (m,))
          w_in : (array_like Shape (n,))  Initial values of parameters of the model
          b_in : (scalar)                 Initial value of parameter of the model
          cost_function:                  function to compute cost
          alpha : (float)                 Learning rate
          num_iters : (int)               number of iterations to run gradient descent
          lambda_ (scalar, float)         regularization constant

        Returns:
          w : (array_like Shape (n,)) Updated values of parameters of the model after
              running gradient descent
          b : (scalar)                Updated value of parameter of the model after
              running gradient descent
        """
        
        w_in = self.w
        b_in = self.b
        for i in range(num_iters):

            # Calculate the gradient and update the parameters
            dj_db, dj_dw = gradient_function(self.X, self.y, w_in, b_in, self.lambda_)   

            # Update Parameters using w, b, alpha and gradient
            w_in = w_in - alpha * dj_dw               
            b_in = b_in - alpha * dj_db              

            # Save cost J at each iteration
            if i<100000:      # prevent resource exhaustion 
                cost =  cost_function(self.X, self.y, w_in, b_in)

            # Print cost every at intervals 10 times or as many iterations if < 10
            if i % show_every == 0 or i == num_iters-1:
                self.J_history.append(cost)
#                 self.w_history.append(self.w)
                print(f"Iteration {i:4}: Cost {float(self.J_history[-1]):8.2f}   ")
        self.w = w_in
        self.b = b_in
        
    
    
    def fit(self, X, y, alpha=0.001, iterations=1500, show_every=None, lambda_= 1):
        """
        setup attributes and apply training
        """
        
        # train data and label
        self.X = X
        self.y = y
        
        # m: number of observations
        self.m, self.n = X.shape

        # regularization coefficient
        self.lambda_ = lambda_
        
        # init weights if first call
        if type(self.w) is not np.ndarray:
            self.w = 0.01 * (np.random.rand(n).reshape(-1,1) - 0.5)
        if self.b == None:
            self.b = -8
        if show_every == None:
            show_every = iterations // 10 
            
        # Perform Gradient Descent
        self.gradient_descent(self.compute_cost, self.compute_gradient, alpha, iterations, show_every)

    
    def predict(self, X, decision_boundary = None): 
        """
        Predict whether the label is 0 or 1 using learned logistic
        regression parameters w

        Args:
        X : (ndarray Shape (m, n))
        w : (array_like Shape (n,))      Parameters of the model
        b : (scalar, float)              Parameter of the model

        Returns:
        p: (ndarray (m,1))
            The predictions for X using a threshold at 0.5
        """
        
        # Check if number of features matches our model
        
        
        # number of training examples
        m, n = X.shape
        p = np.zeros(m)

        f_wb = self.sigmoid(np.dot(X, self.w) + self.b)
        
        if decision_boundary != None:
            for i, prob in enumerate(f_wb):
                p[i] = 1 if prob >= decision_boundary else 0
            return p
        
        return f_wb

In [ ]:
lr = LogisticRegression()

In [ ]:
lr.fit(X_train, y_train[: , 0], alpha=0.00003, iterations=100, lambda_ = 1)

In [ ]:
lr.fit(X_train, y_train[: , 0], alpha=0.000003, iterations=100, lambda_ = 1)

In [ ]:
#Compute accuracy on our training set
p = lr.predict(X_train, 0.5)
print('Train Accuracy: %f'%(np.mean(p == y_train[:, 0]) * 100))

#### Testing...

# Support Multiclass 

In [ ]:
class MultipleLogisticRegression:

    def __init__(self):
    
        self.X = None
        self.y = None
        
        # C: number of categories for Y
        # -> create C models and train them each one at teh time
        self.c = None
        
        self.models = []
        
    def softmax(self, X):
        """
        takes the predicted values from the sub_models
        """
        pred = []
        for i in range(self.c):
            pred.append(self.models[i].predict(X))
        pred = np.array(pred)
        res = []
        for row in pred.T[0]:
            # for each row (student), return the column (house) with the highest probability
            res.append(np.argmax(row / np.sum(row)))
        res = np.array(res)
        return res
    
    def accuracy(self, pred, true_labels):
        """
        calculate the accuracy of the model prediction given the true labels
        """
        if pred.shape != true_labels.shape:
            print('Error in the shape of predictions and true_labels')
            return 
        accuracy = np.mean(pred == true_labels) * 100
        print('Accuracy: %f'%(accuracy))
        return accuracy
    
    def score_matrix(self, pred, true_labels):
        """
        calculate scores matrix
        """
        # Calculate overall precision
        score_matrix = np.zeros((self.c, self.c))
        for i in range(0, len(pred)):
            pred_label = pred[i]
            true_label = answer[i]
            score_matrix[pred_label][true_label] += 1
            
        precision_per_class = []
        for i in range(self.c):
            precision_per_class.append(score_matrix[i][i] / score_matrix[i].sum())
        
        return score_matrix, precision_per_class
        
    def fit(self, X, y, alpha=0.00003, iterations=100, show_every=None, lambda_= 1):
        """
        setup attributes and apply training
        """
        
        # train data and label
        self.X = X
        self.y = y
        
        # m: number of observations
        self.m, self.n = X.shape
        
        # c: number of category
        self.c = y.shape[1]
        
        
        if show_every == None:
            show_every = iterations // 10 
        
        # init models
        if not self.models:
            for i in range(self.c):
                self.models.append(LogisticRegression())
        
        # train models
        decision_boundary = 0.5
        for i in range(self.c):
            model = self.models[i]
            print(f"training model {i+1} with alpha= {0.00003}")
            model.fit(X_train, y_train[: , i], alpha=0.00003, iterations=100, lambda_ = 1)
            p = model.predict(X_train, decision_boundary)
            print('Train Accuracy: %f'%(np.mean(p == y_train[:, i]) * 100))
            
            print(f"training model {i+1} with alpha= {0.00001}")
            model.fit(X_train, y_train[: , i], alpha=0.00001, iterations=100, lambda_ = 1)
            
            p = model.predict(X_train, decision_boundary)
            print('Train Accuracy: %f'%(np.mean(p == y_train[:, i]) * 100))
        
        
    def predict(self, X):
        return self.softmax(X)

In [ ]:
models = MultipleLogisticRegression()

In [ ]:
models.fit(X_train, y_train, lambda_=0.5)

In [ ]:
models.models[1].fit(X_train, y_train[:, 1], alpha=0.00001, iterations=100, lambda_ = 1)

In [ ]:
pred = np.array(models.predict(X_train))

In [ ]:
pred

### Get true labels

In [ ]:
answer = []
for row in y_train:
    answer.append(np.argmax(row))
answer = np.array(answer)

In [ ]:
#Compute accuracy on our training set
print('Train Accuracy: %f'%(np.mean(pred == answer) * 100))

### Score

In [3]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://miro.medium.com/max/1400/1*eS4zV29dd9vFo0KQSk8s7w.png", width=500, height=400)

In [ ]:
score_matrix, precision = models.score_matrix(pred, answer)

In [ ]:
score_matrix

In [ ]:
precision

In [ ]:
print("global precision = ", sum(precision)/len(precision))

## Trying prediction on test set

Reconvert Array of Integer (category number) to 4 arrays of 0and1 for each House

In [ ]:
X_test = np.array(test)
test_pred = models.softmax(X_test)
test_pred

In [ ]:
columns = np.zeros((len(test_pred)))

In [ ]:
df = pd.DataFrame(test_pred, columns=['Hogwarts House'])
target_columns = ['Gryffindor', 'Hufflepuff', 'Ravenclaw', 'Slytherin']

In [ ]:
for i in range(len(df)):
    idx = df['Hogwarts House'][i]
    df['Hogwarts House'][i] = target_columns[idx]

In [ ]:
df['Index'] = df.index

In [ ]:
df

### Save to csv

In [ ]:
df.to_csv('prediction.csv', index=False)